# Shift Planner Modeling Task
The goal is to help a very big restaurant, specialised in on-demand food delivery, to manage its fleet of drivers.

## Problem Statement
The restaurant receives multiple orders every day, and it relies on a sort of forecasting for the next day to estimate how many drivers are required every hour. For example:

Given this demand forecast we need to automatically build the shifts respecting certain constraints:

* Each shift need to be at least of 4h length.
* Each shift cannot be longer than 10h length.
* When we cover the hours of the demand, there might be the case in which we allocate more hours (oversupply). This is an acceptable behavior, but we need to minimise these oversupply hours.

Data
The sample data is available in this dropbox link. It contains some sample cases in JSON format, in which the input includes the forecasted demand and the configuration, such as:
```json
{
  "config": {
    "min_shift_hours": 4,
    "max_shift_hours": 10,
    "timeout_sec": 10
  },
  "timeslots": [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
  "demand": [0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1],
  "solution": {
    "shifts": []
  }
}
```
In the input file, the shifts: [] is obviously empty. The output instead, should be in the same format but with the shifts filled up, as follow:
```json
{
  "config": {
    "min_shift_hours": 4,
    "max_shift_hours": 10,
    "timeout_sec": 10
  },
  "timeslots": [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
  "demand": [0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1],
  "solution": {
    "shifts": [
      [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
      [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
      [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
      [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
      [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]
    ]
  }
}
```
## Evaluation
The solution needs to include:

* __Mathematical formulation of the problem__ (to make it clear and understandable independently of the programming language used).
* __Implementation__ with the preferred programming language, with the instruction to run the given examples (and more).
* __Solutions__ of the forecasted demand shifts in the format defined in the Data section.

The evaluation will consider:

* __Correctness__ of the model described.
* Solution __performance__ (time/quality trade-off).

In case of doubts about the problem statement we require the candidate to make her/his assumptions without being blocked.

In [1]:
# imports

%matplotlib inline

import os
import json
import math
import pprint
import pandas as pd
import matplotlib.pyplot as plt
from itertools import (
    permutations, 
    chain, 
    combinations, 
    combinations_with_replacement)

pp = pprint.PrettyPrinter(indent=4)

In [2]:
# read files

path_to_json = 'planning_demand/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
print(json_files)

['ex2-in.json', 'ex3-out.json', 'ex1-in.json', 'ex3-in.json', 'ex2-out.json', 'ex1-out.json']


In [3]:
# load json data

with open(os.path.join(path_to_json, 'ex1-in.json')) as json_file:
    json_ex1_in = json.load(json_file)
       
with open(os.path.join(path_to_json, 'ex1-out.json')) as json_file:
    json_ex1_out = json.load(json_file)

with open(os.path.join(path_to_json, 'ex2-in.json')) as json_file:
    json_ex2_in = json.load(json_file)

with open(os.path.join(path_to_json, 'ex3-in.json')) as json_file:
    json_ex3_in = json.load(json_file)

__Assumptions__

* We will consider that each driver only works in __one shift__ each day.
* Each shift need to be at least of `4h` length.
* Each shift cannot be longer than `10h` length.
* When we cover the hours of the demand, there might be the case in which we allocate more hours (oversupply). This is an acceptable behavior, but we need to minimise these oversupply hours.

__Approach__

Since each shift needs to be at least `4h` long and at most `10h` long, we can create all possible shift configurations following this rationale:

* __10 hours shift:__ only `3` configurations:
```json
[1,1,1,1,1,1,1,1,1,1,0,0]
[0,1,1,1,1,1,1,1,1,1,1,0]
[0,0,1,1,1,1,1,1,1,1,1,1]
```
* __9 hours shift:__ only `4` configurations:
```json
[1,1,1,1,1,1,1,1,1,0,0,0]
[0,1,1,1,1,1,1,1,1,1,0,0]
[0,0,1,1,1,1,1,1,1,1,1,0]
[0,0,0,1,1,1,1,1,1,1,1,1]
```
* And so on until we reach the 4 hours shift ...

These configurations can be easily generated by obtaining the unique possible permutations between the the block of zeros and block of ones.

For example, we know there are 2 free hours for the `10h` shift (assuming 12 timeslots). Therefore:

```python
# [1] = 10h shift block
permutations(chain.from_iterable([[0,0], [1]]))
# [1,0,0]
# [0,1,0]
# [0,0,1]

# now we can replace the [1] block with 10 working hours
# [1,1,1,1,1,1,1,1,1,1,0,0]
# [0,1,1,1,1,1,1,1,1,1,1,0]
# [0,0,1,1,1,1,1,1,1,1,1,1]
```
We can run these permutations for the 7 different shifts (`4h`, `5h`, `6h`, `7h`, `8h`, `9h` and `10h`) and append all these differents configurations in one array. There are 42 different shift configurations.

If we have the following timeslots and demand forecasted:

```json
{
  "timeslots": [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22],
  "demand": [0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1],
}
```

we can conclude that at least we will need 5 drivers to cover the high-demand timeslots. Consequently, we can obtain all possible combinations with replacement of these 42 configurations of groups in 5 (= drivers).

In this case there are `1,370,754` possible combinations (with repetition of shifts for all 5 drivers). 

Now we need to validate that each timeslot is covered by at least the number of drivers informed by the demand forecast. From the new subset of potential shifts that satisfy the demand it is possible to find the shift configuration that minimizes the total amount of hours (i.e., minimizing oversupply hours).

# Ex1

In [4]:
pp.pprint(json_ex1_in)

{   u'config': {   u'max_shift_hours': 10,
                   u'min_shift_hours': 4,
                   u'timeout_sec': 10},
    u'demand': [0, 1, 1, 1, 1, 5, 5, 5, 5, 5, 5, 1],
    u'solution': {   u'shifts': []},
    u'timeslots': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]}


In [15]:
# Variables.
n_timeslots = len(json_ex1_in['timeslots'])
demand      = json_ex1_in['demand']
max_demand  = max(demand)
l_total     = []

def create_shifts(p, n, l_total):
    '''replaces the shift block with the corresponding working hours'''
    l = []
    for c in p:
        l_item = []
        for item in c:
            if item == 1:
                for i in range(0,n):
                    l_item.append(1)
            else:
                l_item.append(0)
        l.append(l_item)
        
    return append_shifts(l, l_total)

def append_shifts(s, l):
    '''appends each shift to l_total list'''
    for shift in s:
        l.append(shift)

# Creation of all permutations.
for i in range(4, 11):
    p = list(set(permutations(chain.from_iterable([[0]*(n_timeslots-i), [1]]))))
    create_shifts(p, i, l_total)

print len(l_total)

42


In [6]:
m = list(combinations_with_replacement(l_total, max_demand))

In [7]:
print '\nAll possible sets of shifts configurations:', "{:,}".format(len(m)) # check

All possible sets of shifts configurations: 1,370,754


In [8]:
ll = []

def potential_shifts(m, demand, ll):
    """Find all sets of shifts that satisfy demand"""
    for c in m:
        ll.append(c)
        for idx, val in enumerate(demand):
            s = [w[idx] for w in c]
            b = (sum(s)>=val)
            if b == False:
                ll = ll[:-1]
                break
    return ll

ll = potential_shifts(m, demand, ll)

print 'Potential sets of shifts that satisfy demand:', len(ll)

Potential sets of shifts that satisfy demand: 425


In [9]:
ll_m = []
nh   = []

# Calculate total number of hours worked for each set of shifts.
for l in ll:
    n_hours = sum([sum(w) for w in l])
    nh.append(n_hours)
    ll_m.append((n_hours, l))

# Create a dataframe to filter the minimum.
df_shifts = pd.DataFrame(ll_m, columns=['nhours','shift'])

for r in df_shifts[df_shifts['nhours']==df_shifts['nhours'].min()]['shift']:
    pp.pprint(r)


(   [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])


We can compare our result with the one given (`ex1-out.json`)

In [10]:
pp.pprint(json_ex1_out['solution']['shifts'])

[   [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0],
    [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0]]


We have arrived to the same solution following this rationale.

# Ex2 

In [11]:
pp.pprint(json_ex2_in)

{   u'config': {   u'max_shift_hours': 10,
                   u'min_shift_hours': 4,
                   u'timeout_sec': 10},
    u'demand': [0, 9, 9, 9, 16, 16, 17, 17, 17, 17, 17, 17, 6],
    u'solution': {   u'shifts': []},
    u'timeslots': [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]}


In [14]:
# Variables.
n_timeslots = len(json_ex2_in['timeslots'])
demand      = json_ex2_in['demand']
max_demand  = max(demand)
l_total     = []

# Creation of all permutations.
for i in range(4, 11):
    p = list(set(permutations(chain.from_iterable([[0]*(n_timeslots-i), [1]]))))
    create_shifts(p, i, l_total)

print "\n", len(l_total)


49


In [13]:
# the number of items returned
n = 49
r = 17

print "\n", "{:,}".format(math.factorial(n+r-1)/math.factorial(r)/math.factorial(n-1)), "combinations"


1,867,897,112,363,100 combinations


This order of magnitude cannot be handled and it is necessary to find a better approach: __Constraint optimisation__

Please see the second jupyter notebook `optimisation_2` for the implementation of this methodology.